In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.impute import KNNImputer

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
SEED

42

In [3]:
def impute_missing_data(X_with_missing_values):
    imputer = KNNImputer(n_neighbors=10)
    X_imputed = imputer.fit_transform(X_with_missing_values)
    return X_imputed

In [4]:
train_df = pd.read_csv('../new_data/train_set_noisiness_0.2_missingness_0.3.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,1
2,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0
3,0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,1.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0
4,1,0.0,0.0,0.0,NaN,1.0,NaN,NaN,1.0,0.0,...,1.0,3.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0


In [5]:
train_df.fever.value_counts()

0.0    30382
1.0     4898
Name: fever, dtype: int64

In [6]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
cols = X_train.columns
X_train = impute_missing_data(X_train)
X_train, y_train = np.array(X_train), np.array(y_train)
filled_df = pd.DataFrame(X_train, columns=cols)
filled_df.fever.value_counts()

0.0    34681
1.0     4898
0.1     4181
0.2     3576
0.3     2022
0.4      775
0.5      214
0.6       48
0.7        5
Name: fever, dtype: int64

In [7]:
condition1 = (filled_df >= 0.5) & (filled_df < 1)
condition2 = (filled_df > 0) & (filled_df < 0.5)
filled_df[condition1] = 1
filled_df[condition2] = 0
filled_df['label'] = y_train
filled_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
filled_df.fever.value_counts()

0.0    45235
1.0     5165
Name: fever, dtype: int64

In [9]:
filled_df.to_csv('../new_data/knn_imputed/default_mean_less_greater_0.5_k_10/noisiness_0.2_missingness_0.3.csv', index=False)